In [ ]:
!pip install tabula-py

In [ ]:
import tabula
import pandas as pd
from google.colab import data_table
from datetime import datetime,timedelta

In [ ]:
from urllib import request

In [ ]:
file_name = 'Espelho de Ponto.PDF'
dfs = tabula.read_pdf('/content/'+file_name)
infos = dfs[1]
carga_trabalho = timedelta(hours=10.0,minutes=48)
data_table.enable_dataframe_formatter()

'pages' argument isn't specified.Will extract only from page 1 by default.


In [ ]:
uteis = list(filter(lambda dia: 'Normal' in dia,infos["DataTipo de Dia"]))
pontos = list(filter(lambda ponto: str(ponto) != 'nan' and ponto.count(':') >= 1, infos["Ent1Sai1Ent2Sai2Ent3Sai3Ent4Sai4"]))

In [ ]:
def get_dia(data):
  return (data.split(' ')[0], data.split(' ')[2][:3])

In [ ]:
def get_horary(hours):
  return (hours[:5], hours[5:])

In [ ]:
def get_interval_time(time1,time2):
  if time1:
    if time2:
      return datetime.strptime(time2,"%H:%M")-datetime.strptime(time1,"%H:%M")
    else: return "Não bateu" 
  else: return "Não bateu"

In [ ]:
def get_situation(time1,time2):
  if time1:
    if time2:
      if carga_trabalho.seconds-get_interval_time(time1, time2).seconds >= 0: return 'OK'
      else: return 'Carga de trabalho inferior'
      pass
    else: 'Não bateu ponto de saída'
  else: return 'Não bateu ponto de entrada'


In [ ]:
def relatorio():
  relatorio = list()

  for data,hora in zip(uteis,pontos):
    dia, dia_semana = get_dia(data)
    entrada, saida =  get_horary(hora)
    worked_hours = get_interval_time(entrada,saida)
    
    relatorio.append({
        "Data": dia,
        "Dia da semana": dia_semana,
        "Entrada":entrada,
        "Saida": saida,
        "Horas trabalhadas": str(worked_hours),
        "Status": get_situation(entrada,saida)
    })
  display(pd.DataFrame(relatorio))


In [ ]:
relatorio()